In [127]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import time
import os, requests

In [ ]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
     # clck login
    try:
        # log_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="site-nav__account"]'))) 
        login = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "Login"]')))
        if login:
            login.click()
    except Exception as e:
        print("no login")

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Sultanmirza1501#"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
            # driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
    except Exception as e:
        print(f"No check tab found and the error is {e}")
    
    
    # number of cars
    try:
        # time.sleep(1)
        cars = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-4 motorvehicle-pagination-block md-align-right mt"]/p)[1]'))).text.strip("()")
        if cars:
            # print(cars)
            splited_cars = cars.split(" ")
            total_cars = int(splited_cars[5])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
    
    
    # result
    results = []
    car_count = 0
    visited_pages = set()
    while car_count < total_cars: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="col-md-3 col-xs-5"]')))
            
            for i in range(len(page_cars)): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="col-md-3 col-xs-5"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}  
                    
                    # title and lot --> done
                    try:
                        tit_lot = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-title"]'))).text.strip()
                        if tit_lot:
                            tit_lot_splited = tit_lot.split(":")
                            details['Title'] = tit_lot_splited[1]
                            details['Lot'] = tit_lot_splited[0]
                        else:
                            print("No car title and lot number")
                    except Exception as e:
                        print("No car title and lot number")
                        
                    # images --> done
                    try:
                        img_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="slick-list draggable"]'))) 
                        if img_card:
                            imgs =WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))) 
                            if imgs:
                                imgs_lst = [img.get_attribute("src") for img in imgs]
                                details['Images'] = ", ".join(imgs_lst)
                            else:
                                print("No images")
                    except Exception as e:
                        print(f"No image card {e}")
                    
                    # car basic dets
                    try:
                        dets_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="container"])[3]')))
                        if dets_main:
                            
                            # all details --> done
                            try:
                                det1_body = dets_main.find_elements(By.XPATH, './/table[@class="table table-condensed vehicle-table"]')
                                if det1_body:
                                    for det1_b in det1_body:
                                        det1_trs = WebDriverWait(det1_b, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                        for det1_tr in det1_trs:
                                            det1_lbls = det1_tr.find_elements(By.TAG_NAME, 'th')
                                            det1_lbl = det1_lbls[0].text.strip() if det1_lbls else None
                                            
                                            det1_vals = det1_tr.find_elements(By.TAG_NAME, 'td')
                                            det1_val = det1_vals[0].text.strip() if det1_vals else None
                                            if det1_lbl or det1_val:
                                                details[det1_lbl] = det1_val 
                            except Exception as e:
                                print("No det1")
                            
                            
                            try:
                                danger = WebDriverWait(dets_main, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="panel panel-danger"]')))
                                if danger:
                                    dang_head = danger.find_element(By.XPATH, './/div[@class="panel-heading"]').text.strip()
                                    dang_val = danger.find_element(By.TAG_NAME, 'strong').text.strip()
                                    if dang_head and dang_val:
                                        details[dang_head] = dang_val
                                else:
                                    print("No danger tag found")
                            except Exception as e:
                                print("No danger data")
                            
                            
                            # service hist
                            try:
                                serv = dets_main.find_element(By.XPATH, ".//h2[text()='Service History']/following-sibling::div[contains(@class, 'row mb+')][1]")
                                if serv:
                                    serv_trs = WebDriverWait(serv, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="col-md-4"]')))
                                    for serv_tr in serv_trs:
                                        serv_lbls = serv_tr.find_elements(By.TAG_NAME, 'label')
                                        serv_lbl = serv_lbls[0].text.strip() if serv_lbls else None
                                        
                                        serv_vals = serv_tr.find_elements(By.TAG_NAME, 'p')
                                        serv_val = serv_vals[0].text.strip() if serv_vals else None
                                        if serv_lbl or serv_val:
                                            details[serv_lbl] = serv_val 
                            except Exception as e:
                                print("No serv hist")
                                
                            
                            # features --> done
                            try:
                                fets = dets_main.find_element(By.XPATH, ".//h2[text()='Features']/following-sibling::div[contains(@class, 'row mb+')][1]")
                                if fets:
                                    fet_lst = []
                                    # for fet in fets:
                                    fet_trs = WebDriverWait(fets, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                    for fet_tr in fet_trs:
                                        fet_vals = fet_tr.find_element(By.TAG_NAME, 'td').text.strip()
                                        fet_lst.append(fet_vals)
                                    details['Features'] = fet_lst
                            except Exception as e:
                                print("No fets")
                            
                            
                            # cond images
                            try:
                                cond_card = WebDriverWait(dets_main, 10).until(EC.presence_of_element_located((By.XPATH, ".//h2[text()='Condition Check']/following-sibling::div[contains(@class, 'images gallerywidget-condition gallerywidget mb')]")))
                                driver.execute_script("arguments[0].scrollIntoView();", cond_card)
                                time.sleep(3)
                                if cond_card:
                                    cond_imgs = cond_card.find_elements(By.TAG_NAME, 'img')
                                    if cond_imgs:
                                        cond_img_lst = [cond_img.get_attribute("src") for cond_img in cond_imgs]
                                        details['Cond_images'] = cond_img_lst
                            except Exception as e:
                                print("No cond images")                            
                       
                    except Exception as e:
                        print(f"No car dets found")
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
                    
                except Exception as e:
                    print("No car to clk")
                
            if car_count % 30 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                try:
                    next_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="col-md-6 motorvehicle-pagination-block md-align-right"]'))) 
                    next_link = WebDriverWait(next_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/a[contains(@class, "btn") and not(contains(@class, "disabled"))]')))
                    if next_link:
                        for nxt_btn in next_link:
                            page_url = nxt_btn.get_attribute("href")
                            if page_url and page_url not in visited_pages:  # Check if the page has not been visited
                                # print(f"Visiting: {page_url}")
                                visited_pages.add(page_url)  # Mark it as visited
                                page_url.click()
                        
                                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="col-md-3 col-xs-5"]')))
                                print("Hit the next button")
                                # break
                except Exception as e:
                    print("No next found")
                    break
        
        except Exception as e:
            print("No more cars")
            break
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("cag.csv", index=False)
    time.sleep(2)
    driver.quit()
path = "https://www.cityauctiongroup.com/auction/1643"
scarpe(path)

36 number of cars found
No danger data
No serv hist
No danger data
No danger data
No serv hist
No danger data


In [142]:
df = pd.read_csv("cag.csv")
df['Cond_images'][2]

"['https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722325891_fK4fZS.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722325891_fK4fZS.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722325961_hghdQ3.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722326091_whC1PS.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722326133_VEyAPw.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722326212_NLdhdk.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722326382_MwkowV.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722326455_aW8pCq.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722326517_xuNEsa.jpg', 'https://www.cityauctiongroup.com/motorvehicledamageimage/112363/1722326673_Tq0qDD.jpg']"